# Techno-economic analysis

[TEA](../TEA.txt) objects can perform cashflow analysis on a [System](../System.txt) object and arguments for cashflow analysis. These include arguments such as operating days, lang factor, and income tax, as well as arguments for taking into account startup operation, construction schedule, and capital cost financing.

### Inhereting from TEA objects

Depending on the rigour and flexibility of the techno-economic analysis, different parameters may be needed to calculate total depreciable capital (TDC), fixed capital investment (FCI), and fixed operating cost (FOC). For this reason, the TEA object is left as an *abstract class* with *abstract methods* `_TDC`, `_FCI`, and `_FOC`. Here is an example TEA subclass from the lipidcane biorefinery design for the co-production of ethanol and biodiesel:

In [1]:
import biosteam as bst

class LipidcaneTEA(bst.TEA):
    """
    Create a LipidcaneTEA object for techno-economic analysis of a biorefinery [1]_.
    
    Parameters
    ----------    
    system : System
        Should contain feed and product streams.
    IRR : float
        Internal rate of return (fraction).
    duration : tuple[int, int]
        Start and end year of venture (e.g. (2018, 2038)).
    depreciation : str
        'MACRS' + number of years (e.g. 'MACRS7').
    operating_days : float
        Number of operating days per year.
    income_tax : float
        Combined federal and state income tax rate (fraction).
    lang_factor : float
        Lang factor for getting fixed capital investment from
        total purchase cost. If no lang factor, estimate capital investment
        using bare module factors.
    startup_schedule : tuple[float]
        Startup investment fractions per year 
        (e.g. (0.5, 0.5) for 50% capital investment in the first year and 50%
        investment in the second).
    WC_over_FCI : float
        Working capital as a fraction of fixed capital investment.
    labor_cost : float
        Total labor cost (USD/yr).
    fringe_benefits : float
        Cost of fringe benefits as a fraction of labor cost.
    property_tax : float
        Fee as a fraction of fixed capital investment.
    property_insurance : float
        Fee as a fraction of fixed capital investment.    
    supplies : float
        Yearly fee as a fraction of labor cost.
    maintenance : float
        Yearly fee as a fraction of fixed capital investment.
    administration : float
        Yearly fee as a fraction of fixed capital investment.

    References
    ----------
    .. [1] Huang, H., Long, S., & Singh, V. (2016). Techno-economic analysis of biodiesel
        and ethanol co-production from lipid-producing sugarcane. Biofuels, Bioproducts
        and Biorefining, 10(3), 299–315. https://doi.org/10.1002/bbb.1640
    
    """
    
    def __init__(self, system, IRR, duration, depreciation, income_tax,
                 operating_days, lang_factor, construction_schedule, WC_over_FCI,
                 labor_cost, fringe_benefits, property_tax,
                 property_insurance, supplies, maintenance, administration):
        # Huang et. al. does not take into account financing or startup
        # so these parameters are 0 by default
        super().__init__(system, IRR, duration, depreciation, income_tax,
                         operating_days, lang_factor, construction_schedule,
                         startup_months=0, startup_FOCfrac=0, startup_VOCfrac=0,
                         startup_salesfrac=0, finance_interest=0, finance_years=0, 
                         finance_fraction=0, WC_over_FCI=WC_over_FCI)
        self.labor_cost = labor_cost
        self.fringe_benefits = fringe_benefits
        self.property_tax = property_tax
        self.property_insurance = property_insurance
        self.supplies= supplies
        self.maintenance = maintenance
        self.administration = administration
        
    # The abstract _DPI method should take installed equipment cost
    # and return the direct permanent investment. Huang et. al. assume 
    # these values are equal
    def _DPI(self, installed_equipment_cost):
        return installed_equipment_cost
    
    # The abstract _TDC method should take direct permanent investment
    # and return the total depreciable capital. Huang et. al. assume 
    # these values are equal
    def _TDC(self, DPI):
        return DPI
    
    # The abstract _FCI method should take total depreciable capital
    # and return the fixed capital investment. Again, Huang et. al.  
    # assume these values are equal.
    def _FCI(self, TDC):
        return TDC
    
    # The abstract _FOC method should take fixed capital investment
    # and return the fixed operating cost.
    def _FOC(self, FCI):
        return (FCI*(self.property_tax + self.property_insurance
                     + self.maintenance + self.administration)
                + self.labor_cost*(1+self.fringe_benefits+self.supplies))

### Perform cash flow analysis

Create a TEA object from a system:

In [2]:
from biorefineries import lipidcane as lc

lipidcane_tea = LipidcaneTEA(system=lc.lipidcane_sys,
                             IRR=0.15,
                             duration=(2018, 2038),
                             depreciation='MACRS7',
                             income_tax=0.35,
                             operating_days=200,
                             lang_factor=3,
                             construction_schedule=(0.4, 0.6),
                             WC_over_FCI=0.05,
                             labor_cost=2.5e6,
                             fringe_benefits=0.4,
                             property_tax=0.001,
                             property_insurance=0.005,
                             supplies=0.20,
                             maintenance=0.01,
                             administration=0.005)

lipidcane_tea.show() # Print TEA summary at current options
# Ignore the warnings, these are taken care of internally.

LipidcaneTEA: lipidcane_sys
 NPV: 37,554,945 USD at 15.0% IRR


C:\Users\yrc2\OneDrive\Code\biosteam\biosteam\units\design_tools\pressure_vessel.py:85: UserWarning: vacuum pressure vessel ASME codes not implemented yet; wall thickness may be inaccurate and stiffening rings may be required
  warn('vacuum pressure vessel ASME codes not implemented yet; '


Retrieve complete cashflow analysis as a DataFrame object:

In [3]:
lipidcane_tea.get_cashflow_table()

,Depreciable capital [MM$],Fixed capital investment [MM$],Working capital [MM$],Depreciation [MM$],Loan [MM$],...,Net earnings [MM$],Cash flow [MM$],Discount factor,Net present value (NPV) [MM$],Cumulative NPV [MM$]
2016,81.1,81.1,0,0,0,...,0,-81.1,1.15,-93.3,-93.3
2017,122,122,10.1,0,0,...,0,-132,1,-132,-225
2018,0,0,0,29,0,...,15.9,44.9,0.87,39,-186
2019,0,0,0,49.7,0,...,2.47,52.1,0.756,39.4,-147
2020,0,0,0,35.5,0,...,11.7,47.2,0.658,31,-116
2021,0,0,0,25.3,0,...,18.3,43.6,0.572,24.9,-90.7
2022,0,0,0,18.1,0,...,23,41.1,0.497,20.4,-70.3
2023,0,0,0,18.1,0,...,23,41.1,0.432,17.8,-52.5
2024,0,0,0,18.1,0,...,23,41.1,0.376,15.5,-37.1
2025,0,0,0,9.05,0,...,28.9,37.9,0.327,12.4,-24.7


### Find production cost, price, and IRR

Find production cost:

In [4]:
products = [bst.main_flowsheet('ethanol'), bst.main_flowsheet('biodiesel')]
lipidcane_tea.production_cost(products)# USD/yr

array([25294700.708, 33268020.969])

Solve for the price of a stream at the break even point:

In [5]:
feed = bst.main_flowsheet('lipidcane')
lipidcane_tea.solve_price(feed) # USD/kg

0.04011184620563792

Solve for the IRR at the break even point:

In [6]:
lipidcane_tea.IRR = lipidcane_tea.solve_IRR()
lipidcane_tea.show()

LipidcaneTEA: lipidcane_sys
 NPV: 3 USD at 18.0% IRR


Note that the cashflow table is created up to date:

In [7]:
lipidcane_tea.get_cashflow_table()

,Depreciable capital [MM$],Fixed capital investment [MM$],Working capital [MM$],Depreciation [MM$],Loan [MM$],...,Net earnings [MM$],Cash flow [MM$],Discount factor,Net present value (NPV) [MM$],Cumulative NPV [MM$]
2016,81.1,81.1,0,0,0,...,0,-81.1,1.18,-95.7,-95.7
2017,122,122,10.1,0,0,...,0,-132,1,-132,-228
2018,0,0,0,29,0,...,15.9,44.9,0.847,38.1,-190
2019,0,0,0,49.7,0,...,2.47,52.1,0.718,37.5,-152
2020,0,0,0,35.5,0,...,11.7,47.2,0.609,28.7,-123
2021,0,0,0,25.3,0,...,18.3,43.6,0.516,22.5,-101
2022,0,0,0,18.1,0,...,23,41.1,0.437,18,-82.9
2023,0,0,0,18.1,0,...,23,41.1,0.37,15.2,-67.7
2024,0,0,0,18.1,0,...,23,41.1,0.314,12.9,-54.8
2025,0,0,0,9.05,0,...,28.9,37.9,0.266,10.1,-44.7


### System report

Save stream tables, utility requirements, itemized costs, TEA results, and a cash flow table:

In [8]:
# Try this on your computer and open excel
lc.lipidcane_sys.save_report('Lipidcane report.xlsx')
# Ignore the warning. The flowsheet is saved on the excel file
# as a really big image and Python thinks it could be a 
# malicious file cause its so big.